In [7]:
import os 
import pandas as pd
import re

In [8]:
train_csv_path, test_csv_path = "../dataset/csv/label.csv"   , "../dataset/csv/label_test_BCD.csv"
testBCD_csv_path = ["../dataset/csv/label_test_B.csv", "../dataset/csv/label_test_C.csv", "../dataset/csv/label_test_D.csv"]
train_csv, test_csv = pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)
testBCD = [pd.read_csv(csv_path) for csv_path in testBCD_csv_path]
testBCD

[                              img  label
 0     B00019-3R-UM-P3_11-04-N.bmp      1
 1        B00017-4R-P8_15-03-N.bmp      1
 2      B00020-3R-UN-P0_2-01-N.bmp      0
 3     B00004-6L-UM-P1_16-02-N.bmp      1
 4     B00019-3R-UM-P3_11-03-N.bmp      1
 ..                            ...    ...
 203   B00022-6L-UM-P2_13-02-N.bmp      1
 204   B00010-2R-UM-P5_21-10-T.bmp      1
 205     B00012-4L-UN-P0_0-2-T.bmp      0
 206   B00009-3L-UM-P4_11-12-N.bmp      1
 207  B00003-6L-UM-P4_9-1-T-03.bmp      1
 
 [208 rows x 2 columns],
                            img  label
 0    C0004-6L-UM-P3_7-08-T.bmp      1
 1    C0012-6R-UM-P1_1-01-T.bmp      1
 2    C0004-6L-UM-P3_7-02-T.bmp      1
 3    C0004-6L-UM-P2_3-06-T.bmp      1
 4    C0002-6L-UM-P2_4-05-N.bmp      1
 ..                         ...    ...
 57  C0009-6R-UM-P8_18-07-T.bmp      1
 58   C0004-6R-UM-P4_6-05-T.bmp      1
 59  C0003-6L-UM-P2_12-03-T.bmp      1
 60   C0010-6R-UM-P0_5-04-T.bmp      0
 61   C0004-6L-UM-P3_7-04-T.bmp      1
 

In [9]:
train_dict = train_csv.to_dict('list')
test_dict = test_csv.to_dict('list')
testBCD_dict = [test.to_dict('list') for test in testBCD]

In [10]:
x=[a.value_counts("label") for a in testBCD]
x

[label
 1    177
 0     31
 dtype: int64,
 label
 1    47
 0    15
 dtype: int64,
 label
 1    52
 0    45
 dtype: int64]

In [11]:
train_img, train_label = train_dict["img"], train_dict["label"]
test_img, test_label = test_dict["img"], test_dict["label"]
testBCD_img, testBCD_label = [test["img"] for test in testBCD_dict], [test["label"] for test in testBCD_dict]
len(testBCD_img[0])

208

In [17]:
def criterion_sum(test_preds, test_label):
    pattern_po = ".*UM.*"
    pattern_ne = ".*UN.*"
    TP, TN, FP, FN = 0, 0, 0, 0
    for i in range(len(test_preds)):
        if re.match(pattern_po, test_preds[i]):
            if test_label[i] == 1:
                #doctor judge as positive
                TP += 1
            else:
                FP += 1
        elif re.match(pattern_ne, test_preds[i]):
            if test_label[i] == 0:
                TN += 1
            else:
                FN += 1
    recall_po = (TP) / (TP + FN)
    recall_ne = (TN) / (TN + FP)
    f1 = 2*(recall_ne*recall_po)/(recall_ne + recall_po)
    return recall_po, recall_ne, f1

def TN_sum(test_name):
    pattern_t = ".*T.*"
    pattern_n = ".*N.*"
    T, N = 0, 0
    for img in test_name:
        if re.match(pattern_t, img):
            T += 1
        elif re.match(pattern_n, img):
            N += 1
    print(T , " ", N)
    return T, N
            
    

In [18]:
[TN_sum(img) for img in testBCD_img]


38   170
57   5
61   36


[(38, 170), (57, 5), (61, 36)]

In [13]:
criterion_sum(train_img, train_label)


(0.7894025079911483, 0.4344969199178645, 0.5604920640964466)

In [14]:
criterion_sum(test_img, test_label)
# len(test_img)


(0.9630996309963099, 0.4175824175824176, 0.5825721753942847)

In [10]:
[criterion_sum(img, label) for img, label in zip(testBCD_img, testBCD_label)]

[(0.9418604651162791, 0.8709677419354839, 0.9050279329608939),
 (1.0, 0.06666666666666667, 0.125),
 (1.0, 0.2222222222222222, 0.3636363636363636)]